In [2]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


# Problem 1: Math and Extraction with Zero and One-Shot Prompting

The following is the zero shot prompting, where I have not provided any examples to the API.
Loaded the openai key as environment variable and called it by using the os commands.
Instantiated the OpenAPI client and defined the API key.
In the zero shot promot, used two roles, system and user.
In the system role, I defined the assistant asking to extract information from email and output it as JSON
In the User role, I mentioned clearly the input that I would be giving, the format of output that I would be expecting and passed the variable where I have stored the email content.
Temperature has been made 0 for deterministic output.  I do not want any hallucinations here.


In [25]:
import os
import requests
import json
import openai
#import openai
from openai import OpenAI
openai.api_key =os.getenv("openai_api_key")
client = OpenAI()
# Define the email content
email_content = """
Hey Jim,

I'd like to order some new shoes. Please ship the following:

1. Nike Air Jordan I - 2 pair
2. Converse All-Star - 10 pair
3. New Balance 990 - 1 pair
4. Nike Zoom Fly 5 men's red - 2 pair

Please provide sub-totals and grand total cost.

Thanks.

Lance Gentry
123 Main St.
Chelsea, MI 48109
248-229-2229
"""

# Define the zero-shot prompt
messages = [
    {"role": "system", "content": "You are a helpful assistant that extracts information from emails and outputs it as a JSON object."},
    {"role": "user", "content": f"Extract all the information from the following email and format it as a JSON object:\n\n{email_content}"}
]

# Call the OpenAI API
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,  
)

# Print the JSON result
print(response)


ChatCompletion(id='chatcmpl-Anoxfu0zx1iTAnvaueqWzZA76fV65', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "email": {\n    "sender": "Lance Gentry",\n    "recipient": "Jim",\n    "subject": "Order for new shoes",\n    "body": {\n      "order": [\n        {\n          "item": "Nike Air Jordan I",\n          "quantity": 2\n        },\n        {\n          "item": "Converse All-Star",\n          "quantity": 10\n        },\n        {\n          "item": "New Balance 990",\n          "quantity": 1\n        },\n        {\n          "item": "Nike Zoom Fly 5 men\'s red",\n          "quantity": 2\n        }\n      ],\n      "request": "Please provide sub-totals and grand total cost."\n    },\n    "sender_contact": {\n      "address": "123 Main St.",\n      "city": "Chelsea",\n      "state": "MI",\n      "zip": "48109",\n      "phone": "248-229-2229"\n    },\n    "closing": "Thanks."\n  }\n}\n```', refusal=None, role='assistant',

I have processed the response received and used the regex to clean out the initial ''' quotes and the word json to produce the desired json output.

In [28]:
#print(response.choices[0].message.content)
import re
response_content = response.choices[0].message.content  
cleaned_json = response_content.strip("```").lstrip("json").strip()
print(cleaned_json)



{
  "email": {
    "sender": "Lance Gentry",
    "recipient": "Jim",
    "subject": "Order for new shoes",
    "body": {
      "order": [
        {
          "item": "Nike Air Jordan I",
          "quantity": 2
        },
        {
          "item": "Converse All-Star",
          "quantity": 10
        },
        {
          "item": "New Balance 990",
          "quantity": 1
        },
        {
          "item": "Nike Zoom Fly 5 men's red",
          "quantity": 2
        }
      ],
      "request": "Please provide sub-totals and grand total cost."
    },
    "sender_contact": {
      "address": "123 Main St.",
      "city": "Chelsea",
      "state": "MI",
      "zip": "48109",
      "phone": "248-229-2229"
    },
    "closing": "Thanks."
  }
}


For the next prompt, I am giving more clear instructions like the following one to get the outputs in the desired labels.  The name of the labels are different from the one it generated previously.

"Extract all the information from the following email and format it as a JSON object and in addition it should have the following fields shoe_agent_name (this is Jim), request_type shoe_brand, shoe_model, shoe_quantity,customer_name,customer_street,customer_city,customer_state,customer_zip,customer_phone"

In [32]:
import os
import requests
import json
import openai
#import openai
from openai import OpenAI
openai.api_key =os.getenv("openai_api_key")
client = OpenAI()
# Define the email content
email_content = """
Hey Jim,

I'd like to order some new shoes. Please ship the following:

1. Nike Air Jordan I - 2 pair
2. Converse All-Star - 10 pair
3. New Balance 990 - 1 pair
4. Nike Zoom Fly 5 men's red - 2 pair

Please provide sub-totals and grand total cost.

Thanks.

Lance Gentry
123 Main St.
Chelsea, MI 48109
248-229-2229
"""

# Define the zero-shot prompt
messages = [
    {"role": "system", "content": "You are a helpful assistant that extracts information from emails and outputs it as a JSON object."},
    {"role": "user", "content": f"Extract all the information from the following email and format it as a JSON object and in addition it should have the following fields shoe_agent_name (this is Jim), request_type shoe_brand, shoe_model, shoe_quantity,customer_name,customer_street,customer_city,customer_state,customer_zip,customer_phone :\n\n{email_content}"}
]

# Call the OpenAI API
response1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,  
)

# Print the JSON result
print(response1)


ChatCompletion(id='chatcmpl-AnpHECcTh5UdvDv6T5XUCLJuRDjzH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "shoe_agent_name": "Jim",\n  "request_type": "order",\n  "shoes": [\n    {\n      "shoe_brand": "Nike",\n      "shoe_model": "Air Jordan I",\n      "shoe_quantity": 2\n    },\n    {\n      "shoe_brand": "Converse",\n      "shoe_model": "All-Star",\n      "shoe_quantity": 10\n    },\n    {\n      "shoe_brand": "New Balance",\n      "shoe_model": "990",\n      "shoe_quantity": 1\n    },\n    {\n      "shoe_brand": "Nike",\n      "shoe_model": "Zoom Fly 5 men\'s red",\n      "shoe_quantity": 2\n    }\n  ],\n  "customer_name": "Lance Gentry",\n  "customer_street": "123 Main St.",\n  "customer_city": "Chelsea",\n  "customer_state": "MI",\n  "customer_zip": "48109",\n  "customer_phone": "248-229-2229"\n}\n```', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736438544, model=

In [34]:
import re
response1_content = response1.choices[0].message.content  
cleaned1_json = response1_content.strip("```").lstrip("json").strip()
print(cleaned1_json)


{
  "shoe_agent_name": "Jim",
  "request_type": "order",
  "shoes": [
    {
      "shoe_brand": "Nike",
      "shoe_model": "Air Jordan I",
      "shoe_quantity": 2
    },
    {
      "shoe_brand": "Converse",
      "shoe_model": "All-Star",
      "shoe_quantity": 10
    },
    {
      "shoe_brand": "New Balance",
      "shoe_model": "990",
      "shoe_quantity": 1
    },
    {
      "shoe_brand": "Nike",
      "shoe_model": "Zoom Fly 5 men's red",
      "shoe_quantity": 2
    }
  ],
  "customer_name": "Lance Gentry",
  "customer_street": "123 Main St.",
  "customer_city": "Chelsea",
  "customer_state": "MI",
  "customer_zip": "48109",
  "customer_phone": "248-229-2229"
}


For the next prompt, I am giving I am adding the instructions which says the cost of each pair of shoes for different brands and also asking AI to give me the breakdown of the cost and the final grand total cost.

Extract all the information from the following email and format it as a JSON object and in addition it should have the following fields shoe_agent_name (this is Jim), request_type shoe_brand, shoe_model, shoe_quantity,customer_name,customer_street,customer_city,customer_state,customer_zip,customer_phone.  In addition to this also calculate the shoe subtotal and grandtotal cost.  The cost of one pair of Air Jordan I is $100, All-Star is $200, 990 is $300 and Zoom Fly 5 men's red is $400

In [46]:
import os
import requests
import json
import openai
#import openai
from openai import OpenAI
openai.api_key =os.getenv("openai_api_key")
client = OpenAI()
# Define the email content
email_content = """
Hey Jim,

I'd like to order some new shoes. Please ship the following:

1. Nike Air Jordan I - 2 pair
2. Converse All-Star - 10 pair
3. New Balance 990 - 1 pair
4. Nike Zoom Fly 5 men's red - 2 pair

Please provide sub-totals and grand total cost.

Thanks.

Lance Gentry
123 Main St.
Chelsea, MI 48109
248-229-2229
"""

# Define the zero-shot prompt
messages = [
    {"role": "system", "content": "You are a helpful assistant that extracts information from emails and outputs it as a JSON object."},
    {"role": "user", "content": f"Extract all the information from the following email and format it as a JSON object and in addition it should have the following fields shoe_agent_name (this is Jim), request_type shoe_brand, shoe_model, shoe_quantity,customer_name,customer_street,customer_city,customer_state,customer_zip,customer_phone.  In addition to this also calculate the shoe subtotal and grandtotal cost.  The cost of one pair of Air Jordan I is $100, All-Star is $200, 990 is $300 and Zoom Fly 5 men's red is $400:\n\n{email_content}"}
]

# Call the OpenAI API
response2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,  
)

# Print the JSON result
print(response2)


ChatCompletion(id='chatcmpl-Anr8Z6zH1NrMYuMkWdRe1H85I9WZX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "shoe_agent_name": "Jim",\n  "request_type": "order",\n  "shoe_brand": [\n    "Nike",\n    "Converse",\n    "New Balance",\n    "Nike"\n  ],\n  "shoe_model": [\n    "Air Jordan I",\n    "All-Star",\n    "990",\n    "Zoom Fly 5 men\'s red"\n  ],\n  "shoe_quantity": [\n    2,\n    10,\n    1,\n    2\n  ],\n  "customer_name": "Lance Gentry",\n  "customer_street": "123 Main St.",\n  "customer_city": "Chelsea",\n  "customer_state": "MI",\n  "customer_zip": "48109",\n  "customer_phone": "248-229-2229",\n  "shoe_subtotal": {\n    "Air Jordan I": 200,\n    "All-Star": 2000,\n    "990": 300,\n    "Zoom Fly 5 men\'s red": 800\n  },\n  "grand_total_cost": 3300\n}\n```', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736445695, model='gpt-4o-mini-2024-07-18', object='chat.completi

In [47]:
response2_content = response2.choices[0].message.content  
cleaned2_json = response2_content.strip("```").lstrip("json").strip()
print(cleaned2_json)

{
  "shoe_agent_name": "Jim",
  "request_type": "order",
  "shoe_brand": [
    "Nike",
    "Converse",
    "New Balance",
    "Nike"
  ],
  "shoe_model": [
    "Air Jordan I",
    "All-Star",
    "990",
    "Zoom Fly 5 men's red"
  ],
  "shoe_quantity": [
    2,
    10,
    1,
    2
  ],
  "customer_name": "Lance Gentry",
  "customer_street": "123 Main St.",
  "customer_city": "Chelsea",
  "customer_state": "MI",
  "customer_zip": "48109",
  "customer_phone": "248-229-2229",
  "shoe_subtotal": {
    "Air Jordan I": 200,
    "All-Star": 2000,
    "990": 300,
    "Zoom Fly 5 men's red": 800
  },
  "grand_total_cost": 3300
}


Following is the one shot prompt, where I have given an example email and the output I am expecting, including the labels and method of calculation and the format it needs to display.  In particular, I am stressing AI to have the reordering of the ouput.
Then the original email which needs to be converted is given and asked to produce the result.

In [93]:
import os
import requests
import json
import openai
#import openai
from openai import OpenAI
openai.api_key =os.getenv("openai_api_key")
client = OpenAI()
# Define the email content
email_content_ex = """
Hey Harry,

I'd like to order some new shoes. Please ship the following:
1. Nike Air Jordan I - 2 pair
2. Converse All-Star - 20 pair
3. New Balance 990 - 2 pair
4. Nike Zoom Fly 5 men's red - 5 pair

Please provide sub-totals and grand total cost.

Thanks.
Peter
1 cat Blvd.
Tampa, FL 75024
(111) 484-1029
"""
email_content = """
Hey Michelle,

I'd like to order some new shoes. Please ship the following:
1. Nike Air Jordan I - 1 pair
2. Converse All-Star - 20 pair
3. New Balance 990 - 2 pair
4. Nike Zoom Fly 5 men's red - 5 pair

Please provide sub-totals and grand total cost.

Thanks.
Artis Gilmore
723 Lexington Blvd.
New York, NY 10001
(503) 484-1029
"""

# Define the zero-shot prompt
messages = [
    {"role": "system", "content": "You are a helpful assistant that extracts information from emails and outputs it as a JSON object."},
    {"role": "user", "content": f"Extract all the information from the following email {email_content_ex} and format it as a \
    JSON object .  Here is the output format I am expecting, \
     \
    'shoe_agent_name': 'Harry', \
    'request_type': 'Order', \
    'customer_name': 'Peter', \
    'customer_street': '1 cat Blvd.', \
    'customer_city': 'Tampa', \
    'customer_state': 'FL', \
    'customer_zip': 75024, \
    'customer_phone': '(111) 484-1029', \
    \
    'items': [ \
    'shoe_brand': 'Nike', \
    'shoe_model': 'Air Jordan I', \
    'shoe_quantity': 2, \
    'shoe_price': 100, \
    'shoe_subtotal': 200, \
    'shoe_brand': 'Converse', \
    'shoe_model': 'All-Star', \
    'shoe_quantity': 10, \
    'shoe_price': 30, \
    'shoe_subtotal': 300, \
    'shoe_brand': 'New Balance', \
    'shoe_model': '990', \
    'shoe_quantity': 2, \
    'shoe_price': 110, \
    'shoe_subtotal': 220, \
    'shoe_brand': 'Nike', \
    'shoe_model': 'Zoom Fly 5 mens red', \
    'shoe_quantity': 2, \
    'shoe_price': 400, \
    'shoe_subtotal': 800], \
    'grand_total': 1520 \
    \n\n Now, do the same thing for {email_content}"}]


# Call the OpenAI API
response3 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,  
)

# Print the JSON result
print(response3)


ChatCompletion(id='chatcmpl-AnrtxDTH5eCu1JnLpPS5UGZf99CMo', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n    "shoe_agent_name": "Michelle",\n    "request_type": "Order",\n    "customer_name": "Artis Gilmore",\n    "customer_street": "723 Lexington Blvd.",\n    "customer_city": "New York",\n    "customer_state": "NY",\n    "customer_zip": 10001,\n    "customer_phone": "(503) 484-1029",\n    "items": [\n        {\n            "shoe_brand": "Nike",\n            "shoe_model": "Air Jordan I",\n            "shoe_quantity": 1,\n            "shoe_price": 100,\n            "shoe_subtotal": 100\n        },\n        {\n            "shoe_brand": "Converse",\n            "shoe_model": "All-Star",\n            "shoe_quantity": 20,\n            "shoe_price": 30,\n            "shoe_subtotal": 600\n        },\n        {\n            "shoe_brand": "New Balance",\n            "shoe_model": "990",\n            "shoe_quantity": 2,\n       

In [95]:
response3_content = response3.choices[0].message.content  
cleaned3_json = response3_content.strip("```").lstrip("json").strip()
print(cleaned3_json)

{
    "shoe_agent_name": "Michelle",
    "request_type": "Order",
    "customer_name": "Artis Gilmore",
    "customer_street": "723 Lexington Blvd.",
    "customer_city": "New York",
    "customer_state": "NY",
    "customer_zip": 10001,
    "customer_phone": "(503) 484-1029",
    "items": [
        {
            "shoe_brand": "Nike",
            "shoe_model": "Air Jordan I",
            "shoe_quantity": 1,
            "shoe_price": 100,
            "shoe_subtotal": 100
        },
        {
            "shoe_brand": "Converse",
            "shoe_model": "All-Star",
            "shoe_quantity": 20,
            "shoe_price": 30,
            "shoe_subtotal": 600
        },
        {
            "shoe_brand": "New Balance",
            "shoe_model": "990",
            "shoe_quantity": 2,
            "shoe_price": 110,
            "shoe_subtotal": 220
        },
        {
            "shoe_brand": "Nike",
            "shoe_model": "Zoom Fly 5 men's red",
            "shoe_quantity": 5,
    

# Problem 2: Chat Completion for Chain of Thought

In the below code, we are directing the OpenAI to breakdown the problem and think in the series of steps.  
We have explicitly provided examples in the format of Q and A and finally our desired question being asked.
For eg, for the example question "Olivia has $23. She bought five bagels for $3 each. How much money does she have left?".  Calculation has been done step by step, like the cost of each bagel multiplied by the number of bagels and subtracting it from the total money and then arriving at the answer.



In [102]:
import os
import requests
import json
import openai
#import openai
from openai import OpenAI
openai.api_key =os.getenv("openai_api_key")
client = OpenAI()
# Define the email content
content = """
Q: There were nine computers in the server room. Five more computers were installed each day, from Monday to Thursday. How many computers are now in the server room?
A: There are 4 days from Monday to Thursday. 5 computers were added each day. That means in total 4 * 5 = 20 computers were added. There were 9 computers initially, so now there are 9 + 20 = 29 computers. The answer is 29.
Q: Michael had 58 golf balls. On Tuesday, he lost 23 golf balls. On Wednesday, he lost 2 more. How many golf balls did he have at the end of Wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent $15. She has $8 left.
Q: When I was 6 my sister was half my age. Now I'm 70 how old is my sister?
"""

# Define the zero-shot prompt
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": f"Learn and answer the question{content}"}
]

# Call the OpenAI API
response4 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,  
)

# Print the JSON result
print(response4)


ChatCompletion(id='chatcmpl-AnuVQJXPBJIxPtkVUil31yN9HqiGH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='When you were 6, your sister was half your age, which means she was 3 years old (6 / 2 = 3). The age difference between you and your sister is 3 years. \n\nNow that you are 70, your sister is 70 - 3 = 67 years old. \n\nThe answer is 67.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736458644, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_01aeff40ea', usage=CompletionUsage(completion_tokens=73, prompt_tokens=293, total_tokens=366, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [103]:
response4_content = response4.choices[0].message.content  
print(response4_content)

When you were 6, your sister was half your age, which means she was 3 years old (6 / 2 = 3). The age difference between you and your sister is 3 years. 

Now that you are 70, your sister is 70 - 3 = 67 years old. 

The answer is 67.


However, what I do not understand is, how the OpenAI is giving correct answer, even if we do not pass the CoT.
Specific message has been passed to forget the previous CoT as well, "You are a helpful assistant, and forget the previous CoT"

In [104]:
import os
import requests
import json
import openai
#import openai
from openai import OpenAI
openai.api_key =os.getenv("openai_api_key")
client = OpenAI()
# Define the email content
content = """
Q: When I was 6 my sister was half my age. Now I'm 70 how old is my sister?
"""

# Define the zero-shot prompt
messages = [
    {"role": "system", "content": "You are a helpful assistant, and forget the previous CoT"},
    {"role": "user", "content": f"Just answer the question{content}"}
]

# Call the OpenAI API
response5 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,  
)

# Print the JSON result
print(response5)


ChatCompletion(id='chatcmpl-AnuVZgBGJUFf6DzGtfQrDNZGgtJgK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your sister is 67 years old.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736458653, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f2cd28694a', usage=CompletionUsage(completion_tokens=9, prompt_tokens=52, total_tokens=61, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
